In [9]:
from io import StringIO
import pandas
import glob

Download NCHS linked data (and discard non-NHIS linked data)

In [ ]:
!wget --recursive ftp://ftp.cdc.gov/pub/Health_Statistics/NCHS/datalinkage/linked_mortality
!mkdir NCHS_linked_data
!mv ./ftp.cdc.gov/pub/Health_Statistics/NCHS/datalinkage/linked_mortality/NHIS_*.dat ./NCHS_linked_data/
!rm -r ./ftp.cdc.gov

Load NCHS linked data from disk

Read in all the NCHS linked mortality datafiles

In [8]:
nchs_file_paths = glob.glob("./NCHS_linked_data/NHIS_*.dat")

In [ ]:
# Sanity check
assert len(nchs_file_paths) > 0, "No NCHS files available!"

In [4]:
nchs_column_widths = [14,1,1,3,1,1,1,4,8,8]
nchs_column_names = ["PUBLICID", "ELIGSTAT", "MORTSTAT", "UCOD_LEADING", "DIABETES", "HYPERTEN", "DODQTR", "DODYEAR", "WGT_NEW", "SA_WGT_NEW"]

nchs_dataframes = [
    pandas.read_fwf(
        file_path,
        widths=nchs_column_widths,
        names=nchs_column_names,
        dtype=False, #{"PUBLICID": "object"}
        na_values=['.']
    )
    for file_path
    in nchs_file_paths
]

In [ ]:
# Sanity check
total_row_count = sum([dataframe.shape[0] for dataframe in nchs_dataframes])
total_line_count = !wc -l ./NCHS_linked_data/NHIS_*.dat | grep total | cut -f2 -d' '
assert total_row_count == total_line_count, "Expected {} to equal {}".format(total_row_count, total_line_count)

In [14]:
# Debug info
for dataframe in nchs_dataframes:
   print("{:8d}".format(dataframe.shape[0]))

print("=" * 8)
print("{:8d}".format(total_row_count))

NameError: name 'nchs_dataframes' is not defined

Combine NCHS dataframes into one big dataframe

In [5]:
nchs_data = pandas.concat(nchs_nhis_linked_dataframes)

In [ ]:
# Sanity check
separate_row_count = sum([dataframe.shape[0] for dataframe in nchs_dataframes])
combined_row_count = nchs_data.shape[0]
assert separate_row_count == combined_row_count, "Expected {} to equal {}".format(separate_row_count, combined_row_count)

Read in 

In [ ]:
# Set the path of your NHIS data extract (csv)
nhis_file_path = "../NHIS/nhis_test.csv.gz"

In [7]:
nhis_chunks = pandas.read_csv(
    nhis_file_path,
    compression="gzip",
    chunksize=100000
)

Join the NCHS linked data with the raw NHIS data

In [ ]:
#chunk = nhis_chunks.get_chunk()

```python
pandas.merge(
    restaurant_ids_dataframe,
    restaurant_review_frame,
    on='business_id',
    how='outer',
    suffixes=('_restaurant_id', '_restaurant_review')
)
```